# End-to-End RAG with Kamiwaza SDK (Offsets-Only)

This notebook demonstrates a complete RAG flow using the Kamiwaza SDK without storing chunk text in the vector database. The flow:

1. Discover Markdown documents in `notebooks/sdk/`
2. Register them in the catalog
3. Chunk with precise byte offsets and embed
4. Insert vectors with offsets-only metadata into Milvus
5. Enter a query, retrieve top-K relevant chunks by vector search
6. Reconstruct chunk text from source files via offsets and generate an answer using the first active deployed model

Notes:
- You’ll need the Kamiwaza API running locally and a deployment available for the LLM step.
- Offsets-only storage keeps Milvus lean and uses the source files as the single source of truth.


In [ ]:
# Configuration
import os
import time
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple

from kamiwaza_sdk import kamiwaza_sdk as kz

API_URL = os.environ.get("KAMIWAZA_API_URL", "http://localhost:7777/api/")
DOCS_DIR = Path.cwd()  # assume MD files are alongside the notebook

EMBEDDER_MODEL = "BAAI/bge-base-en-v1.5"
PROVIDER_TYPE = "huggingface_embedding"
CHUNK_SIZE = 600
OVERLAP = 102
TOP_K = 5
COLLECTION_NAME = f"SDKRAG_{int(time.time())}"

client = kz(API_URL)
print(f"API: {API_URL}")
print(f"Docs dir: {DOCS_DIR}")
print(f"Collection: {COLLECTION_NAME}")

In [ ]:
# Utilities
from typing import Optional


def list_markdown_files(root: Path, max_files: int = 1000) -> List[Path]:
    files: List[Path] = []
    if root.is_file():
        if root.suffix.lower() == ".md":
            return [root]
        return []
    for p in root.rglob("*.md"):
        if ".ipynb_checkpoints" in p.parts:
            continue
        files.append(p)
        if len(files) >= max_files:
            break
    return files


def safe_read_utf8_window(path: Path, start: int, length: int) -> str:
    try:
        with open(path, "rb") as f:
            f.seek(max(0, start))
            data = f.read(max(0, length))
        try:
            return data.decode("utf-8")
        except UnicodeDecodeError:
            return data.decode("utf-8", errors="ignore")
    except Exception as exc:
        return f"<error reading bytes: {exc}>"


def rechunk_get_chunk_text(
    client: kz,
    model: str,
    provider_type: str,
    source_path: Path,
    offset: int,
    chunk_size: int,
    overlap: int,
) -> Optional[str]:
    try:
        text = source_path.read_text(encoding="utf-8")
    except Exception:
        return None

    embedder = client.embedding.get_embedder(
        model=model,
        provider_type=provider_type,
    )
    resp = embedder.chunk_text(
        text=text,
        max_length=chunk_size,
        overlap=overlap,
        return_metadata=True,
    )

    chunks = resp.chunks if hasattr(resp, "chunks") else resp.get("chunks", [])
    offsets = resp.offsets if hasattr(resp, "offsets") else resp.get("offsets", [])

    for ch_text, ch_off in zip(chunks, offsets or []):
        if int(ch_off) == int(offset):
            return ch_text
    return None


In [ ]:
# 1) Discover & Catalog
files = list_markdown_files(DOCS_DIR)
if not files:
    raise ValueError(f"No markdown files found in {DOCS_DIR}")

print(f"Found {len(files)} docs")

path_to_urn: Dict[Path, str] = {}
for fp in files:
    ds = client.catalog.create_dataset(
        dataset_name=str(fp.resolve()),
        platform="file",
        environment="PROD",
        description=f"RAG demo: {fp.name}",
    )
    path_to_urn[fp] = ds.urn
    print(f"Cataloged: {fp.name} -> {ds.urn}")


In [ ]:
# 2) Chunk, Embed, and Insert (Offsets-Only)
embedder = client.embedding.get_embedder(model=EMBEDDER_MODEL, provider_type=PROVIDER_TYPE)

inserted_total = 0
print(f"Collection: {COLLECTION_NAME}")

for fp in files:
    print(f"\nProcessing: {fp.name}")
    text = fp.read_text(encoding="utf-8")

    # chunk with offsets
    resp = embedder.chunk_text(text=text, max_length=CHUNK_SIZE, overlap=OVERLAP, return_metadata=True)
    chunks = resp.chunks if hasattr(resp, "chunks") else resp["chunks"]
    offsets = resp.offsets if hasattr(resp, "offsets") else resp.get("offsets", [])
    print(f" - chunks: {len(chunks)}")

    # embed
    vectors = embedder.embed_chunks(chunks)

    # metadata (autofields only)
    metadata = []
    for off in (offsets or [0]*len(chunks)):
        metadata.append({
            "model_name": EMBEDDER_MODEL,
            "source": str(fp.resolve()),
            "catalog_urn": path_to_urn[fp],
            "offset": int(off),
            "filename": fp.name,
        })

    # insert
    client.vectordb.insert(
        vectors=vectors,
        metadata=metadata,
        collection_name=COLLECTION_NAME,
        field_list=None,
    )

    print(f" - inserted: {len(chunks)}")
    inserted_total += len(chunks)

print(f"\nInserted total: {inserted_total}")


In [ ]:
# 3) Interactive Query → Retrieve → Preview

def retrieve(query: str, k: int = TOP_K):
    q_vec = embedder.create_embedding(query).embedding
    hits = client.vectordb.search(
        query_vector=q_vec,
        collection_name=COLLECTION_NAME,
        limit=k,
        output_fields=["source", "offset", "filename", "catalog_urn", "model_name"],
    )
    results = []
    for h in hits:
        score = getattr(h, "score", None)
        if score is None and isinstance(h, dict):
            score = h.get("score", 0.0)
        meta = getattr(h, "metadata", None) or (h.get("metadata") if isinstance(h, dict) else {})
        results.append({"score": float(score or 0.0), "metadata": meta})
    return results


def preview_hits(hits, mode="rechunk", pre_bytes=500, post_bytes=2000):
    print(f"Found {len(hits)} results\n")
    for i, item in enumerate(hits, 1):
        meta = item["metadata"]
        source = meta.get("source")
        offset = int(meta.get("offset", 0))
        fname = meta.get("filename")
        score = item.get("score", 0.0)
        print(f"{i:02d}. score={score:.4f} file={fname} offset={offset}")
        if not source or not Path(source).exists():
            print("   <missing source>")
            continue
        if mode == "rechunk":
            text = rechunk_get_chunk_text(client, EMBEDDER_MODEL, PROVIDER_TYPE, Path(source), offset, CHUNK_SIZE, OVERLAP)
            if text is None:
                text = safe_read_utf8_window(Path(source), max(0, offset - pre_bytes), pre_bytes + post_bytes)
        elif mode == "around":
            text = safe_read_utf8_window(Path(source), max(0, offset - pre_bytes), pre_bytes + post_bytes)
        else:  # from
            text = safe_read_utf8_window(Path(source), offset, post_bytes)
        print("--- preview ---")
        print(text)
        print()

# Example (you can re-run this cell):
# hits = retrieve("How does Kamiwaza benefit the enterprise?", k=TOP_K)
# preview_hits(hits, mode="rechunk")


In [ ]:
# 4) Generate with the first active deployment

def pick_first_active_deployment_name(client: kz) -> Optional[str]:
    try:
        deps = client.serving.list_active_deployments()
    except Exception:
        deps = []
    if not deps:
        return None
    dep = deps[0]
    return getattr(dep, "m_name", None) or getattr(dep, "name", None)


def call_llm_with_context(question: str, context: str) -> str:
    model_name = pick_first_active_deployment_name(client)
    if not model_name:
        return "<No active deployments found>\n" + context
    openai_client = client.openai.get_client(model_name)
    system_prompt = (
        "You are a helpful assistant. Answer using ONLY the provided context. "
        "If not in context, say you do not know."
    )
    user_content = f"Question:\n{question}\n\nContext:\n{context}"
    try:
        resp = openai_client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content},
            ],
            timeout=600,
        )
        return resp.choices[0].message.content
    except Exception as exc:
        return f"<LLM error: {exc}>\n" + context

# Example workflow
# query = "How does Kamiwaza benefit the enterprise?"
# hits = retrieve(query, k=TOP_K)
# preview_hits(hits)
# ctx, cits = build_context = (lambda *args, **kwargs: None), None


In [ ]:
# 5) Run an example query end-to-end (adjust the query and re-run)
query = "How does Kamiwaza benefit the enterprise?"

hits = retrieve(query, k=TOP_K)
#preview_hits(hits, mode="rechunk")

# Build context for the LLM
# Reuse the same logic as previews; cap to ~8k chars
ctx_parts = []
for i, item in enumerate(hits, 1):
    meta = item["metadata"]
    src = meta.get("source")
    off = int(meta.get("offset", 0))
    if not src or not Path(src).exists():
        continue
    text = rechunk_get_chunk_text(client, EMBEDDER_MODEL, PROVIDER_TYPE, Path(src), off, CHUNK_SIZE, OVERLAP)
    if text is None:
        text = safe_read_utf8_window(Path(src), max(0, off - 500), 2500)
    header = f"[Source {i}] {Path(src).name} @ offset {off}"
    ctx_parts.append(f"{header}\n---\n{text}\n")

context = "\n\n".join(ctx_parts)[:8000]

answer = call_llm_with_context(query, context)
print("\n===== RAG ANSWER =====\n")
print(answer)
print("\n======================\n")
